In [1]:
import pandas as pd
import numpy as np


In [2]:
#task_name_list = ['dod','dod_3day','dod_7day','dod_30day','dod_60day','dod_90day','DNR','Weaning_successful','SBT']
task_name_list = ['dod','dod_3day','dod_7day','dod_30day','dod_60day','dod_90day','DNR','Weaning_successful','SBT','dialysis','InvasiveVent','Vasopressor']

#task_name_list = ['dialysis','InvasiveVent','Vasopressor']
#data_path = "./data/sample"

data_path = "C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample"

In [3]:
all_task_data = {}

for task_name in task_name_list:
    
    # read
    train_X = np.load(f"{data_path}/train_X_{task_name}.npy", allow_pickle=True)
    train_Y = np.load(f"{data_path}/train_Y_{task_name}.npy", allow_pickle=True)
    validation_X = np.load(f"{data_path}/validation_X_{task_name}.npy", allow_pickle=True)
    validation_Y = np.load(f"{data_path}/validation_Y_{task_name}.npy", allow_pickle=True)
    test_X = np.load(f"{data_path}/test_X_{task_name}.npy", allow_pickle=True)
    test_Y = np.load(f"{data_path}/test_Y_{task_name}.npy", allow_pickle=True)
    
    print(f'{task_name} ==> {train_X.shape}')
    print(f'{task_name} ==> {train_Y.shape}')
    print(f'{task_name} ==> {validation_X.shape}')
    print(f'{task_name} ==> {validation_Y.shape}')
    print(f'{task_name} ==> {test_X.shape}')
    print(f'{task_name} ==> {test_Y.shape}')
    print("******************")

    
    all_task_data[task_name] = {
        'train': {'X': train_X[:,:], 'X_with_id': train_X, 'Y': train_Y},
        'validation': {'X': validation_X[:,:], 'X_with_id': validation_X,'Y': validation_Y},
        'test': {'X': test_X[:,:], 'X_with_id': test_X, 'Y': test_Y}
    }
    

dod ==> (9572, 3, 271)
dod ==> (9572,)
dod ==> (1197, 3, 271)
dod ==> (1197,)
dod ==> (1197, 3, 271)
dod ==> (1197,)
******************
dod_3day ==> (11203, 3, 271)
dod_3day ==> (11203,)
dod_3day ==> (1400, 3, 271)
dod_3day ==> (1400,)
dod_3day ==> (1401, 3, 271)
dod_3day ==> (1401,)
******************
dod_7day ==> (12617, 3, 271)
dod_7day ==> (12617,)
dod_7day ==> (1577, 3, 271)
dod_7day ==> (1577,)
dod_7day ==> (1578, 3, 271)
dod_7day ==> (1578,)
******************
dod_30day ==> (15132, 3, 271)
dod_30day ==> (15132,)
dod_30day ==> (1891, 3, 271)
dod_30day ==> (1891,)
dod_30day ==> (1892, 3, 271)
dod_30day ==> (1892,)
******************
dod_60day ==> (16236, 3, 271)
dod_60day ==> (16236,)
dod_60day ==> (2030, 3, 271)
dod_60day ==> (2030,)
dod_60day ==> (2030, 3, 271)
dod_60day ==> (2030,)
******************
dod_90day ==> (16752, 3, 271)
dod_90day ==> (16752,)
dod_90day ==> (2094, 3, 271)
dod_90day ==> (2094,)
dod_90day ==> (2095, 3, 271)
dod_90day ==> (2095,)
******************
DNR ==

In [4]:
num_features = all_task_data[task_name]['train']['X'].shape[2]

#串接所有sample input
combined_X = np.empty((0,num_features))
for task_name in task_name_list:
    for data_type in ['train','validation','test']:
        x = all_task_data[task_name][data_type]['X']
        reshaped_x = np.reshape(x, (-1,x.shape[2]))  # (sample*seq, feature)
        combined_X = np.vstack((combined_X, reshaped_x))

print(combined_X.shape)

(561270, 271)


# Standardization

In [5]:
from sklearn.preprocessing import MinMaxScaler
import joblib

#計算轉換器
scaler = MinMaxScaler(feature_range=(0, 1)).fit(combined_X[:,:])

#保存轉換器
#joblib.dump(scaler, './data/scaler_model.joblib')

joblib.dump(scaler, 'C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/scaler_model.joblib')

#讀取轉換器
#scaler = joblib.load('./data/scaler_model.joblib')
#input()

['C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/scaler_model.joblib']

In [6]:
def scale_data(data, scaler):
    sample_count = data.shape[0]
    seq_count = data.shape[1]
    #print(data.shape)
    assert seq_count == 3
    reshape_data = np.reshape(data, (-1,data.shape[2]))
    scalar_data = scaler.transform(reshape_data)
    return np.reshape(scalar_data, (sample_count, seq_count, data.shape[2]))

In [7]:
for task_name in task_name_list:
    # 對訓練集進行縮放
    all_task_data[task_name]['train']['scalar_X'] = scale_data(all_task_data[task_name]['train']['X'][:,:], scaler)
    # 對驗證集進行縮放
    all_task_data[task_name]['validation']['scalar_X'] = scale_data(all_task_data[task_name]['validation']['X'][:,:], scaler)
    # 對測試集進行縮放
    all_task_data[task_name]['test']['scalar_X'] = scale_data(all_task_data[task_name]['test']['X'][:,:], scaler)
    

In [8]:
for task_name in task_name_list:
    train_y = all_task_data[task_name]['train']['Y']
    val_y = all_task_data[task_name]['validation']['Y']
    test_y = all_task_data[task_name]['test']['Y']
    
    total = np.concatenate((train_y, val_y), axis=0)
    total = np.concatenate((total, test_y), axis=0)
    one_count = np.count_nonzero(total == 1)
    zero_count = np.count_nonzero(total == 0)
    
    print(f'Task:{task_name} => 1:{one_count}({round(one_count*100/(one_count+zero_count),2)}%)   0:{zero_count}({round(zero_count*100/(one_count+zero_count),2)}%)')

Task:dod => 1:1048(8.76%)   0:10918(91.24%)
Task:dod_3day => 1:3675(26.24%)   0:10329(73.76%)
Task:dod_7day => 1:5872(37.23%)   0:9900(62.77%)
Task:dod_30day => 1:9930(52.5%)   0:8985(47.5%)
Task:dod_60day => 1:11681(57.55%)   0:8615(42.45%)
Task:dod_90day => 1:12540(59.88%)   0:8401(40.12%)
Task:DNR => 1:5119(33.47%)   0:10177(66.53%)
Task:Weaning_successful => 1:2164(43.98%)   0:2756(56.02%)
Task:SBT => 1:4748(54.01%)   0:4043(45.99%)
Task:dialysis => 1:3668(25.46%)   0:10740(74.54%)
Task:InvasiveVent => 1:19800(71.85%)   0:7759(28.15%)
Task:Vasopressor => 1:3752(26.38%)   0:10470(73.62%)


# Save

In [9]:
#save_path = './data/sample_new/standard_data/'
#save_path = './data/sample/standard_data/'

save_path = 'C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample/standard_data/'

In [10]:
import os
for task_name in task_name_list: #[Weaning、SBT、 ...]
    for data_type in all_task_data[task_name].keys():  #train、validation、test
        for data_name in all_task_data[task_name][data_type].keys():  #X、Y、scalar_X
            file_name = f'{data_type}_{data_name}_{task_name}'
            folder = os.path.join(save_path, file_name)
            data = all_task_data[task_name][data_type][data_name]
            np.save(f'{folder}.npy', data)
print('finish')

finish
